### QNN (Q-Wertbasiertes Neuronales Netzwerk)

Für die Erweiterbarkeit unseres Codes ist uns aufgefallen, dass der Q-Algorithmus seine Grenzen bei Spielen mit einem beinahe unendlichen Zustandsraum aufweist. Spiele wie Schach oder Go haben einen derart riesigen Zustandsraum aller möglichen Züge, der ohne weitere Hilfe nicht einfach durch ausprobieren komplett erkundet werden kann. Um dem Prinzip des bestärkenden Lernens nahe zu kommen, müssen andere Methoden gefunden werden, um zukünftige Züge oder sogar Strategien bei unendlich wirkenden Zustandsräumen hervorzusagen. Hierfür soll ein neuronales Netzwerk mit mehreren Schichten zum Einsatz kommen.

Der Gedanke dahinter ist, dass man nicht mehr versucht alle Zustände zu erkunden und perfekt vorherzusagen, sondern eine Struktur oder sogar Strategie in gewissen Zügen zu erkennen. Der Fokus des neuronalen Netzes soll damit sein, Strukturen in zeitlich aufeinanderfolgender Züge zu erkennen und bei unbekannten Zuständen einen Schätzwert auf Basis bisher bekannter Strategien ausgeben.

Ein neuronales Netzwerk besteht zumeist aus Eingabevektoren, verschiedenste versteckte Schichten sowie Ausgabevektoren. Genauso wie die Q-Funktion, sollen dem QNN gewisse Parameter als Eingabevektoren mitgegeben werden. Dazu gehört der aktuelle Zustand des Bretts, die ausgewählte Aktion auf Basis vorheriger Werte aus dem QNN Modell und ggf. die Belohnung. Als Ausgabevektor soll, genauso wie bei der Q-Funktion, ein Q-Wert sein, der die maximale Belohnung des aktuellen Zustand-Aktion-Paares beschreibt.

Die Lerndaten erstellt der QNN Agent selbst, durch das explorative Erkunden mittels dem Explorationsfaktor namens "Theta". Anhand der explorierten Daten und erlangten Belohnungen wird das QNN selbst die besten Züge herausfinden.

In [4]:
from build.Board import Board
from build.player.qlearner.QLearner import QLearner

import random
import numpy as np

In [5]:
class QNNLearner(QLearner):
    """
    QLearner specification for neural network learning
    """

    def __init__(self, model = None, learn_rate=0.1, discount_factor=0.8, batch_size=10, learn_online=False):
        """
        :param model: used qnn model structure
        :param learn_rate: learning rate of this q learner
        :param discount_factor: discount factor of this q learner
        """
        self.learn_rate = learn_rate
        self.model = model
        self.discount_factor = discount_factor
        self.batch_size = batch_size
        self.learn_online = learn_online
        self.model.online = True if learn_online else False

    def update(self, prev_state, state, prev_move, reward):
        """
        Update q players knowledge by learning offline or online
        :param prev_state: previous known state
        :param state: new state
        :param prev_move: previous made move
        :param reward: previous reward
        """
        if self.learn_online:
            # Online training
            self.model.train_model_online(prev_state, prev_move, state, reward)
        else:
            # Offline training
            self.model.train_model_offline(prev_state, prev_move, state, reward)


    def select_move(self, state, theta=0.1) -> int:
        """
        Select the best move or, if exploring, a random move
        :param state: current state
        :param theta: temperature value (optional)
        :return: chosen action
        """
        p = random.uniform(0, 1)

        if p > theta:
            action = self.model.predict_model(state, self.learn_online)
        else:
            action = random.choice(list(Board.POSSIBLE_ACTIONS))

        return action # return choosen move

    def is_known_state(self, state) -> bool:
        """
        Does nothing because it doesn't make sense for a qnn learner. QNN should not store previous states,
        instead the network learns patterns of states and corresponding actions.
        """
        raise RuntimeError('State reduction is not applicable for qnn!')

von Kevin Bücher